In [1]:
import mysql.connector
import pandas as pd
from mysql.connector import connect, Error

In [ ]:
fetch_weather = """select * from weather
                    where weather.weatherdate >= "2018-01-01" and weather.weatherdate < "2019-01-01";"""

In [ ]:
## from https://realpython.com/python-mysql/
try:
    with connect(
        host="localhost",
        user="team11",
        password=os.environment["DB_PASS"],
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [66]:
wdf = pd.read_csv("weather_all.csv")

In [67]:
wdf.head()

,weatherdate,dateind,rain,rainind,temp,tempind,wetb,dewpt,vappr,rhum,msl
0,2018-01-01 00:00:00,0,0.0,0,4.6,0,3.5,1.8,6.9,82,991.0
1,2018-01-01 01:00:00,0,0.1,0,4.7,0,3.6,1.8,7.0,81,991.1
2,2018-01-01 02:00:00,0,0.0,0,4.8,0,3.7,1.9,7.0,81,991.1
3,2018-01-01 03:00:00,0,0.0,0,4.9,0,3.8,2.2,7.2,82,990.7
4,2018-01-01 04:00:00,0,0.0,0,5.3,0,4.1,2.3,7.2,81,990.3


In [68]:
wdf.tail()

,weatherdate,dateind,rain,rainind,temp,tempind,wetb,dewpt,vappr,rhum,msl
8755,2018-12-31 19:00:00,0,0.0,0,9.9,0,7.9,5.5,9.0,74,1034.9
8756,2018-12-31 20:00:00,0,0.0,0,9.9,0,8.0,5.8,9.2,75,1035.0
8757,2018-12-31 21:00:00,0,0.0,0,9.9,0,7.9,5.7,9.1,75,1035.0
8758,2018-12-31 22:00:00,0,0.0,0,9.9,0,8.0,5.9,9.3,76,1035.1
8759,2018-12-31 23:00:00,0,0.0,0,9.8,0,7.9,5.7,9.1,75,1035.2


In [69]:
print("Number of rows:" ,wdf.shape[0])
print("Number of columns:" ,wdf.shape[1])

Number of rows: 8760
Number of columns: 11


In [70]:
wdf.dtypes

weatherdate     object
dateind          int64
rain           float64
rainind          int64
temp           float64
tempind          int64
wetb           float64
dewpt          float64
vappr          float64
rhum             int64
msl            float64
dtype: object

In [71]:
wdf.columns

Index(['weatherdate', 'dateind', 'rain', 'rainind', 'temp', 'tempind', 'wetb',
       'dewpt', 'vappr', 'rhum', 'msl'],
      dtype='object')

In [72]:
datetime_columns = wdf[['weatherdate']].columns
continuous_columns = wdf[['dateind', 'rain', 'rainind', 'temp', 'tempind', 'wetb',
       'dewpt', 'vappr', 'rhum', 'msl']].columns

for column in continuous_columns:
    wdf[column] = wdf[column].astype('float64')  

for column in datetime_columns:
    wdf[column] = wdf[column].astype('datetime64[ns]')  
    
wdf.dtypes

weatherdate    datetime64[ns]
dateind               float64
rain                  float64
rainind               float64
temp                  float64
tempind               float64
wetb                  float64
dewpt                 float64
vappr                 float64
rhum                  float64
msl                   float64
dtype: object

In [73]:
wdfT = wdf.T
print("Number of duplicate (excluding first) columns in the table is: ", wdfT.duplicated().sum())
print("Number of duplicate (including first) columns in the table is: ",  wdfT[wdfT.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) columns in the table is:  2
Number of duplicate (including first) columns in the table is:  3


In [74]:
wdf.select_dtypes(['float64']).describe().T

,count,mean,std,min,25%,50%,75%,max
dateind,8760.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
rain,8760.0,0.078208,0.342674,0.0,0.0,0.0,0.0,8.6
rainind,8760.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
temp,8760.0,10.449932,5.653197,-4.5,6.3,10.0,14.5,27.5
tempind,8760.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
wetb,8760.0,8.503311,4.679926,-4.6,5.1,8.4,12.0,20.4
dewpt,8760.0,6.377043,4.650843,-9.8,3.1,6.5,9.7,18.3
vappr,8760.0,10.031336,3.147217,2.9,7.6,9.7,12.1,21.0
rhum,8760.0,77.064384,14.043860,24.0,68.0,80.0,88.0,99.0
msl,8760.0,1013.114235,11.762356,979.5,1005.7,1014.9,1021.6,1041.7


In [75]:
wdf.select_dtypes(include=['datetime64[ns]']).describe().T

,count,unique,top,freq,first,last
weatherdate,8760,8760,2018-08-01 12:00:00,1,2018-01-01,2018-12-31 23:00:00


In [76]:
print('Number of duplicate (excluding first) rows in the table is: ', wdf.duplicated().sum())
print('Number of duplicate rows (including first) in the table is:', wdf[wdf.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [77]:
wdf.isnull().sum()

weatherdate    0
dateind        0
rain           0
rainind        0
temp           0
tempind        0
wetb           0
dewpt          0
vappr          0
rhum           0
msl            0
dtype: int64

In [78]:
for feature in continuous_columns:
    print(feature)
    print("----------\n")
    print("Range {} is: ".format(feature), (wdf[feature].max() - wdf[feature].min()))
    print("----------")
    print('{0:.5}  {1}'.format("Value", "Number of Instances"))
    print(wdf[feature].value_counts().nlargest(15), "\n\n\n")


dateind
----------

Range dateind is:  0.0
----------
Value  Number of Instances
0.0    8760
Name: dateind, dtype: int64 



rain
----------

Range rain is:  8.6
----------
Value  Number of Instances
0.0    7570
0.1     373
0.2     166
0.3     108
0.4      94
0.5      74
0.6      55
0.7      48
0.8      39
0.9      27
1.0      24
1.3      18
1.1      17
1.4      14
1.8      14
Name: rain, dtype: int64 



rainind
----------

Range rainind is:  0.0
----------
Value  Number of Instances
0.0    8760
Name: rainind, dtype: int64 



temp
----------

Range temp is:  32.0
----------
Value  Number of Instances
9.4     87
7.3     79
12.1    71
8.9     71
9.9     70
8.6     70
6.1     70
6.4     67
8.4     66
9.5     66
13.1    65
7.5     65
11.9    65
7.7     64
7.8     63
Name: temp, dtype: int64 



tempind
----------

Range tempind is:  0.0
----------
Value  Number of Instances
0.0    8760
Name: tempind, dtype: int64 



wetb
----------

Range wetb is:  25.0
----------
Value  Number of Insta

In [79]:
features_cardinality = list(wdf[continuous_columns].columns.values)

print('{0:35}  {1}'.format("Feature", "Cardinality"))
print('{0:35}  {1}'.format("-------", "--------------- \n"))

for c in features_cardinality:
    print('{0:35}  {1}'.format(c, str(len(wdf[c].unique()))))

Feature                              Cardinality
-------                              --------------- 

dateind                              1
rain                                 44
rainind                              1
temp                                 304
tempind                              1
wetb                                 243
dewpt                                257
vappr                                171
rhum                                 76
msl                                  584


In [80]:
for feature in datetime_columns:
    print(feature)
    print("----------\n")
    print("Range {} is: ".format(feature), (wdf[feature].max() - wdf[feature].min()))
    print("----------")
    print('{0:.5}  {1}'.format("Value", "Number of Instances"))
    print(wdf[feature].value_counts().nlargest(15), "\n\n\n")


weatherdate
----------

Range weatherdate is:  364 days 23:00:00
----------
Value  Number of Instances
2018-08-01 12:00:00    1
2018-10-30 16:00:00    1
2018-01-26 17:00:00    1
2018-06-09 11:00:00    1
2018-09-01 04:00:00    1
2018-02-04 16:00:00    1
2018-09-19 17:00:00    1
2018-02-13 15:00:00    1
2018-09-23 21:00:00    1
2018-02-25 20:00:00    1
2018-02-22 14:00:00    1
2018-12-17 02:00:00    1
2018-11-22 00:00:00    1
2018-03-03 13:00:00    1
2018-05-16 16:00:00    1
Name: weatherdate, dtype: int64 





In [81]:
features_cardinality = list(wdf[datetime_columns].columns.values)

print('{0:35}  {1}'.format("Feature", "Cardinality"))
print('{0:35}  {1}'.format("-------", "--------------- \n"))

for c in features_cardinality:
    print('{0:35}  {1}'.format(c, str(len(wdf[c].unique()))))

Feature                              Cardinality
-------                              --------------- 

weatherdate                          8760


## Weather findings

    - constant columns dateind, rainind and tempind
    - duplicate columns dateind, rainind and tempind
    
# Solutions:
    
    - drop columns dateind, rainind and tempind

In [82]:
wdf = wdf.drop("dateind", 1)
wdf = wdf.drop("rainind", 1)
wdf = wdf.drop("tempind", 1)

In [83]:
wdf.head()

,weatherdate,rain,temp,wetb,dewpt,vappr,rhum,msl
0,2018-01-01 00:00:00,0.0,4.6,3.5,1.8,6.9,82.0,991.0
1,2018-01-01 01:00:00,0.1,4.7,3.6,1.8,7.0,81.0,991.1
2,2018-01-01 02:00:00,0.0,4.8,3.7,1.9,7.0,81.0,991.1
3,2018-01-01 03:00:00,0.0,4.9,3.8,2.2,7.2,82.0,990.7
4,2018-01-01 04:00:00,0.0,5.3,4.1,2.3,7.2,81.0,990.3


In [84]:
wdf.to_csv('weather_clean.csv', index=False)